In [1]:
import pyodbc
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import json
from datetime import date,datetime
import numpy as np

In [2]:
r = requests.get("https://github.com/owid/covid-19-data/tree/master/public/data",allow_redirects=True)
r = r.content

In [3]:
# create soup object
soup = bs(r,"html.parser")

In [4]:
# export html doc to file stored
with open('response.html','wb') as file:
    file.write(soup.prettify('utf-8'))

In [5]:
# search & navigate html tree to extract data
links = soup.find_all('a',class_="js-navigation-open Link--primary",title='owid-covid-data.csv')

In [6]:
#extract text
link = soup.find('a',class_="js-navigation-open Link--primary",title='owid-covid-data.csv')
link.text

'owid-covid-data.csv'

In [7]:
base_url = 'https://raw.githubusercontent.com'

In [8]:
# extract data from html tree to get attribute value of href file link
for i in links:
    r_url = repr(i.get('href'))
    url_1 = base_url.strip("'")+r_url.strip("'")
    url = url_1.strip('/blob')
    print(url)

https://raw.githubusercontent.com/owid/covid-19-data/blob/master/public/data/owid-covid-data.csv


In [9]:
response= requests.get("https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv")
response.status_code

200

In [10]:
open(url.rsplit('/', 1)[1], 'wb').write(response.content)

42149365

In [11]:
url.rsplit('/', 1)[1]

'owid-covid-data.csv'

In [12]:
# r = json.loads(response.content)

In [13]:
# covid_data=pd.DataFrame.from_dict(r,orient='index')
# covid_data.head()

In [14]:
covid_data = pd.read_csv('owid-covid-data.csv')

In [15]:
covid_data.columns

Index(['iso_code', 'continent', 'location', 'date', 'total_cases', 'new_cases',
       'new_cases_smoothed', 'total_deaths', 'new_deaths',
       'new_deaths_smoothed', 'total_cases_per_million',
       'new_cases_per_million', 'new_cases_smoothed_per_million',
       'total_deaths_per_million', 'new_deaths_per_million',
       'new_deaths_smoothed_per_million', 'reproduction_rate', 'icu_patients',
       'icu_patients_per_million', 'hosp_patients',
       'hosp_patients_per_million', 'weekly_icu_admissions',
       'weekly_icu_admissions_per_million', 'weekly_hosp_admissions',
       'weekly_hosp_admissions_per_million', 'new_tests', 'total_tests',
       'total_tests_per_thousand', 'new_tests_per_thousand',
       'new_tests_smoothed', 'new_tests_smoothed_per_thousand',
       'positive_rate', 'tests_per_case', 'tests_units', 'total_vaccinations',
       'people_vaccinated', 'people_fully_vaccinated', 'total_boosters',
       'new_vaccinations', 'new_vaccinations_smoothed',
       't

In [16]:
covid_death = covid_data[['iso_code', 'continent', 'location', 'date', 'total_cases', 'new_cases',
       'new_cases_smoothed', 'total_deaths', 'new_deaths',
       'new_deaths_smoothed', 'total_cases_per_million',
       'new_cases_per_million', 'new_cases_smoothed_per_million',
       'total_deaths_per_million', 'new_deaths_per_million',
       'new_deaths_smoothed_per_million']]
covid_death = covid_death.fillna(value=0)
covid_death.tail().sort_values(by='date', ascending=False)

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million
150394,ZWE,Africa,Zimbabwe,2021-12-27,205449.0,1098.0,1481.429,4908.0,17.0,14.714,13612.952,72.753,98.159,325.202,1.126,0.975
150393,ZWE,Africa,Zimbabwe,2021-12-26,204351.0,605.0,1811.143,4891.0,6.0,15.571,13540.199,40.087,120.005,324.075,0.398,1.032
150392,ZWE,Africa,Zimbabwe,2021-12-25,203746.0,1010.0,2025.571,4885.0,14.0,15.143,13500.112,66.922,134.213,323.678,0.928,1.003
150391,ZWE,Africa,Zimbabwe,2021-12-24,202736.0,1392.0,1881.286,4871.0,16.0,13.143,13433.190,92.233,124.653,322.750,1.060,0.871
150390,ZWE,Africa,Zimbabwe,2021-12-23,201344.0,1940.0,2755.286,4855.0,27.0,13.714,13340.957,128.543,182.564,321.690,1.789,0.909


In [17]:
covid_vaccination = covid_data[['iso_code', 'continent', 'location', 'date','total_vaccinations',
       'people_vaccinated', 'people_fully_vaccinated', 'total_boosters',
       'new_vaccinations', 'new_vaccinations_smoothed',
       'total_vaccinations_per_hundred', 'people_vaccinated_per_hundred',
       'people_fully_vaccinated_per_hundred', 'total_boosters_per_hundred',
       'new_vaccinations_smoothed_per_million',
       'new_people_vaccinated_smoothed',
       'new_people_vaccinated_smoothed_per_hundred']]
covid_vaccination = covid_vaccination.fillna(value=0)
covid_vaccination.tail().sort_values(by='date', ascending=False)

,iso_code,continent,location,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,new_vaccinations,new_vaccinations_smoothed,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,total_boosters_per_hundred,new_vaccinations_smoothed_per_million,new_people_vaccinated_smoothed,new_people_vaccinated_smoothed_per_hundred
150394,ZWE,Africa,Zimbabwe,2021-12-27,7218438.0,4105296.0,3113142.0,0.0,3798.0,11918.0,47.83,27.20,20.63,0.0,790.0,6425.0,0.043
150393,ZWE,Africa,Zimbabwe,2021-12-26,7214640.0,4103288.0,3111352.0,0.0,5591.0,12994.0,47.80,27.19,20.62,0.0,861.0,7018.0,0.047
150392,ZWE,Africa,Zimbabwe,2021-12-25,7209049.0,4100069.0,3108980.0,0.0,8273.0,13872.0,47.77,27.17,20.60,0.0,919.0,7436.0,0.049
150391,ZWE,Africa,Zimbabwe,2021-12-24,7200776.0,4095685.0,3105091.0,0.0,12648.0,15484.0,47.71,27.14,20.57,0.0,1026.0,8135.0,0.054
150390,ZWE,Africa,Zimbabwe,2021-12-23,7188128.0,4089315.0,3098813.0,0.0,20518.0,18522.0,47.63,27.10,20.53,0.0,1227.0,9432.0,0.062


In [18]:
connection = pyodbc.connect('Driver={SQL Server};'
                      'Server=LAPTOP-E28526DD\SQLEXPRESS;'
                      'Database=coviddata;'
                      'Trusted_Connection=yes;')

In [19]:
cursor = connection.cursor()

In [ ]:
# cursor.execute("DROP TABLE IF EXISTS [covid_death$];")
# cursor.execute("DROP TABLE IF EXISTS [covid_vaccination$];")

In [ ]:
# cursor.execute("""
#   CREATE TABLE covid_death (
#     id serial PRIMARY KEY,
#     description VARCHAR NOT NULL
#   );
# """)

In [ ]:
# # initial data load
# for index, row in covid_death.iterrows():
#     cursor.execute("INSERT INTO [coviddata].[dbo].[covid_death$]\
#                    (iso_code, continent, location, date, total_cases, new_cases,new_cases_smoothed, total_deaths, new_deaths\
#                    ,new_deaths_smoothed, total_cases_per_million,new_cases_per_million, new_cases_smoothed_per_million\
#                    ,total_deaths_per_million, new_deaths_per_million,new_deaths_smoothed_per_million) \
#                    values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)" \
#                    ,row.iso_code, row.continent, row.location, row.date, row.total_cases \
#                    ,row.new_cases,row.new_cases_smoothed, row.total_deaths \
#                    ,row.new_deaths,row.new_deaths_smoothed, row.total_cases_per_million \
#                    ,row.new_cases_per_million, row.new_cases_smoothed_per_million,row.total_deaths_per_million \
#                    ,row.new_deaths_per_million,row.new_deaths_smoothed_per_million)
# connection.commit()

In [ ]:
# # initial data load
# for index, row in covid_vaccination.iterrows():
#     cursor.execute("INSERT INTO [coviddata].[dbo].[covid_vaccination$] \
#     (iso_code, continent, location, date,total_vaccinations,people_vaccinated, people_fully_vaccinated \
#                    , total_boosters,new_vaccinations, new_vaccinations_smoothed,total_vaccinations_per_hundred \
#                    , people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred, total_boosters_per_hundred \
#                    , new_vaccinations_smoothed_per_million,new_people_vaccinated_smoothed \
#                    , new_people_vaccinated_smoothed_per_hundred) \
#                    values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)" \
#                    , row.iso_code, row.continent, row.location, row.date  \
#                    , row.total_vaccinations,row.people_vaccinated, row.people_fully_vaccinated  \
#                    , row.total_boosters,row.new_vaccinations, row.new_vaccinations_smoothed \
#                    ,row.total_vaccinations_per_hundred, row.people_vaccinated_per_hundred \
#                    ,row.people_fully_vaccinated_per_hundred, row.total_boosters_per_hundred \
#                    , row.new_vaccinations_smoothed_per_million,row.new_people_vaccinated_smoothed \
#                    , row.new_people_vaccinated_smoothed_per_hundred)
# connection.commit()

In [20]:
# initiate last_update var
last_update = datetime.strptime('2021-12-24',"%Y-%m-%d")
print(last_update)

2021-12-24 00:00:00


In [23]:
# convert date col to datetime type
covid_death.date = covid_death.date.apply(lambda x:datetime.strptime(x,"%Y-%m-%d"))
covid_vaccination.date = covid_vaccination.date.apply(lambda x:datetime.strptime(x,"%Y-%m-%d"))

In [27]:
# compare data diff from last update
covid_death = covid_death[covid_death.date>last_update]
covid_vaccination = covid_vaccination[covid_vaccination.date>last_update]

In [26]:
## insert diff from last_update
for index, row in covid_death.iterrows():
    cursor.execute("INSERT INTO [coviddata].[dbo].[covid_death$]\
                   (iso_code, continent, location, date, total_cases, new_cases,new_cases_smoothed, total_deaths, new_deaths\
                   ,new_deaths_smoothed, total_cases_per_million,new_cases_per_million, new_cases_smoothed_per_million\
                   ,total_deaths_per_million, new_deaths_per_million,new_deaths_smoothed_per_million) \
                   values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)" \
                   ,row.iso_code, row.continent, row.location, row.date, row.total_cases \
                   ,row.new_cases,row.new_cases_smoothed, row.total_deaths \
                   ,row.new_deaths,row.new_deaths_smoothed, row.total_cases_per_million \
                   ,row.new_cases_per_million, row.new_cases_smoothed_per_million,row.total_deaths_per_million \
                   ,row.new_deaths_per_million,row.new_deaths_smoothed_per_million)

connection.commit()

In [28]:
for index, row in covid_vaccination.iterrows():
    cursor.execute("INSERT INTO [coviddata].[dbo].[covid_vaccination$] \
    (iso_code, continent, location, date,total_vaccinations,people_vaccinated, people_fully_vaccinated \
                   , total_boosters,new_vaccinations, new_vaccinations_smoothed,total_vaccinations_per_hundred \
                   , people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred, total_boosters_per_hundred \
                   , new_vaccinations_smoothed_per_million,new_people_vaccinated_smoothed \
                   , new_people_vaccinated_smoothed_per_hundred) \
                   values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)" \
                   , row.iso_code, row.continent, row.location, row.date  \
                   , row.total_vaccinations,row.people_vaccinated, row.people_fully_vaccinated  \
                   , row.total_boosters,row.new_vaccinations, row.new_vaccinations_smoothed \
                   ,row.total_vaccinations_per_hundred, row.people_vaccinated_per_hundred \
                   ,row.people_fully_vaccinated_per_hundred, row.total_boosters_per_hundred \
                   , row.new_vaccinations_smoothed_per_million,row.new_people_vaccinated_smoothed \
                   , row.new_people_vaccinated_smoothed_per_hundred)
connection.commit()

In [31]:
#update last_update
last_update = datetime.today()
print(last_update)

2021-12-28 19:08:10.873897


In [29]:
cursor.execute('SELECT top 5 * FROM [coviddata].[dbo].[covid_vaccination$]')

In [30]:
result = cursor.fetchall() # retrieve previous statement
print(result)

[('AFG', 'Asia', 'Afghanistan', datetime.datetime(2020, 2, 24, 0, 0), None, None, None, None, None, None, None, None, None, '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None), ('AFG', 'Asia', 'Afghanistan', datetime.datetime(2020, 2, 25, 0, 0), None, None, None, None, None, None, None, None, None, '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None), ('AFG', 'Asia', 'Afghanistan', datetime.datetime(2020, 2, 26, 0, 0), None, None, None, None, None, None, None, None, None, '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None), ('AFG', 'Asia', 'Afghanistan', datetime.datetime(2020, 2, 27, 0, 0), None, None, None, Non

In [ ]:
connection.close()
cursor.close()